# Zillow

### Wrangling the Zillow data

#### Aquuires and prepares Zillow data

## Imports

In [1]:
import numpy as np
import pandas as pd

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:.2f}'.format


# Stats
import scipy.stats as stats

# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

#My Files
import env

## Acquire

In [2]:
def get_db_url(database):
    from env import host, user, password
    url = f'mysql+pymysql://{user}:{password}@{host}/{database}'
    return url

# acquire zillow data using the query
def get_zillow():
    # SQL Query
    sql = '''

    SELECT prop.*, 
           pred.logerror, 
           pred.transactiondate, 
           air.airconditioningdesc, 
           arch.architecturalstyledesc, 
           build.buildingclassdesc, 
           heat.heatingorsystemdesc, 
           landuse.propertylandusedesc, 
           story.storydesc, 
           construct.typeconstructiondesc 
    FROM   properties_2017 prop  
           INNER JOIN (SELECT parcelid,
                       Max(transactiondate) transactiondate 
                       FROM   predictions_2017 
  
                       GROUP  BY parcelid) pred 
                   USING (parcelid)
                   
                            JOIN predictions_2017 as pred USING (parcelid, transactiondate)
           LEFT JOIN airconditioningtype air USING (airconditioningtypeid) 
           LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid) 
           LEFT JOIN buildingclasstype build USING (buildingclasstypeid) 
           LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid) 
           LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid) 
           LEFT JOIN storytype story USING (storytypeid) 
           LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid) 
    WHERE  prop.latitude IS NOT NULL 
           AND prop.longitude IS NOT NULL
           AND pred.id IN (SELECT MAX(id)
           FROM predictions_2017
           GROUP BY parcelid
           HAVING MAX(transactiondate));
       
       
'''
    url = get_db_url('zillow')
    zillow_df = pd.read_sql(sql, url, index_col='id')
    return zillow_df


In [3]:
df = get_zillow()

In [4]:
df.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,censustractandblock,logerror,transactiondate,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc
id,,,,,,,,,,,,,,,,,,,,,
1727539,14297519,NaN,NaN,NaN,3.50,4.00,NaN,NaN,3.50,NaN,...,60590630072012.00,0.03,2017-01-01,None,None,None,None,Single Family Residential,None,None
1387261,17052889,NaN,NaN,NaN,1.00,2.00,NaN,NaN,1.00,NaN,...,61110010023006.00,0.06,2017-01-01,None,None,None,None,Single Family Residential,None,None
11677,14186244,NaN,NaN,NaN,2.00,3.00,NaN,NaN,2.00,NaN,...,60590218022012.00,0.01,2017-01-01,None,None,None,None,Single Family Residential,None,None
2288172,12177905,NaN,NaN,NaN,3.00,4.00,NaN,8.00,3.00,NaN,...,60373001001006.00,-0.10,2017-01-01,None,None,None,Central,Single Family Residential,None,None
1970746,10887214,1.00,NaN,NaN,3.00,3.00,NaN,8.00,3.00,NaN,...,60371236012000.00,0.01,2017-01-01,Central,None,None,Central,Condominium,None,None


## Prepare

In [15]:
# a function to drop missing values based on a threshold
def handle_missing_values(df, prop_required_row = 0.5, prop_required_col = 0.5):
    ''' function which takes in a dataframe, required notnull proportions of non-null rows and columns.
    drop the columns and rows columns based on theshold:'''
    
    #drop columns with nulls
    threshold = int(prop_required_col * len(df.index)) # Require that many non-NA values.
    df.dropna(axis = 1, thresh = threshold, inplace = True)
    
    #drop rows with nulls
    threshold = int(prop_required_row * len(df.columns)) # Require that many non-NA values.
    df.dropna(axis = 0, thresh = threshold, inplace = True)
    
    
    return df


In [13]:
def wrangle_zillow():
    # read saved .csv
    df = pd.read_csv('zillow.csv')
    
    # propertylandusetypeid that can be considered "single unit" to df
    single_unit = [261, 262, 263, 264, 268, 273, 275, 276, 279]
    df = df[df.propertylandusetypeid.isin(single_unit)]
    
    # df with bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet > 0
    df = df[(df.bedroomcnt > 0) & (df.bathroomcnt > 0) & (df.calculatedfinishedsquarefeet>0)]

    
    # drop missing values based on a threshold
    df = handle_missing_values(df)
   
    # drop unnecessary columns
    df = df.drop(columns=['calculatedbathnbr', 'finishedsquarefeet12', 'fullbathcnt', 'heatingorsystemtypeid', 'propertyzoningdesc', 'censustractandblock','propertycountylandusecode', 'propertylandusetypeid', 'propertylandusedesc', 'unitcnt','heatingorsystemdesc'])
    
    # drop null rows for specific columns only
    df = df[df.regionidzip.notnull()]
    df = df[df.yearbuilt.notnull()]
    df = df[df.structuretaxvaluedollarcnt.notnull()]
    df = df[df.taxvaluedollarcnt.notnull()]
    df = df[df.landtaxvaluedollarcnt.notnull()]
    df = df[df.taxamount.notnull()]

    # fill NaNs with mode
    df.buildingqualitytypeid.mode()[0]
    df['buildingqualitytypeid'] = df.buildingqualitytypeid.fillna(df.buildingqualitytypeid.mode()[0])
    df.lotsizesquarefeet.mode()[0]
    df['lotsizesquarefeet'] = df.lotsizesquarefeet.fillna(df.lotsizesquarefeet.mode()[0])
    df.regionidcity.mode()[0]
    df['regionidcity'] = df.regionidcity.fillna(df.regionidcity.mode()[0])

    
    # crete column called county that lists column type based on fips
    df['county'] = df['fips'].apply(
    lambda x: 'Los Angeles' if x == 6037\
    else 'Orange' if x == 6059\
    else 'Ventura')
    
    # check for outliers
    df = df[df.taxvaluedollarcnt < 5_000_000]
    df[df.calculatedfinishedsquarefeet < 8000]
    
    
    # drop nulls to make sure none were missed
    df = df.dropna()
    
    return df
    


In [14]:
df = wrangle_zillow()
df.head()

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedfinishedsquarefeet,fips,latitude,longitude,lotsizesquarefeet,...,roomcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,logerror,transactiondate,county
0,1727539,14297519,3.50,4.00,6.00,3100.00,6059.00,33634931.00,-117869207.00,4506.00,...,0.00,1998.00,485713.00,1023282.00,2016.00,537569.00,11013.72,0.03,2017-01-01,Orange
1,1387261,17052889,1.00,2.00,6.00,1465.00,6111.00,34449266.00,-119281531.00,12647.00,...,5.00,1967.00,88000.00,464000.00,2016.00,376000.00,5672.48,0.06,2017-01-01,Ventura
2,11677,14186244,2.00,3.00,6.00,1243.00,6059.00,33886168.00,-117823170.00,8432.00,...,6.00,1962.00,85289.00,564778.00,2016.00,479489.00,6488.30,0.01,2017-01-01,Orange
3,2288172,12177905,3.00,4.00,8.00,2376.00,6037.00,34245180.00,-118240722.00,13038.00,...,0.00,1970.00,108918.00,145143.00,2016.00,36225.00,1777.51,-0.10,2017-01-01,Los Angeles
6,781532,12095076,3.00,4.00,9.00,2962.00,6037.00,34145202.00,-118179824.00,63000.00,...,0.00,1950.00,276684.00,773303.00,2016.00,496619.00,9516.26,-0.00,2017-01-01,Los Angeles
